# Building a PR Article Generation Application using Crew AI

This notebook demonstrates how to build a multi-agent system using CrewAI for automated PR article generation. The system uses a reflection pattern with multiple specialized agents that collaborate to write, review, and refine PR articles.

## Architecture Overview
- **PR Writer Agent**: Creates initial article drafts based on outlines
- **PR Reviewer Agent**: Reviews articles against quality guidelines
- **PR Rewriter Agent**: Refines articles based on feedback
- **Supervisor Agent**: Orchestrates the workflow between agents

The following diagram depicts the overall architecture and the worklfow:

<img src="../../imgs/lab5-crewai-architecture.png" width="800"> 


The system leverages AWS Bedrock for LLM capabilities and Knowledge Bases for retrieving examples of high-quality PR articles.

## Initial Setup and Imports

Import the necessary libraries for CrewAI agents, tasks, and regular expressions for text processing.

In [1]:
from crewai import Agent, Crew, Process, Task
import re

## Configuration and Environment Setup

Load stored variables from previous notebook sessions and set up environment configuration.

In [2]:
%store -r

Set the maximum number of iterations for agent refinement cycles to prevent infinite loops.

In [10]:
max_iterations = 2

## AWS Services Initialization

Set up AWS clients for Bedrock, S3, and other services needed for the agentic workflow. This includes:
- **STS Client**: For getting account identity information
- **S3 Client**: For storing and retrieving knowledge base documents
- **Bedrock Runtime**: For LLM inference
- **Bedrock Agent Runtime**: For knowledge base operations

In [5]:
import boto3
from langchain_aws import ChatBedrockConverse
from typing import Optional

sts_client = boto3.client('sts')
session = boto3.session.Session()

account_id = sts_client.get_caller_identity()["Account"]
region = session.region_name

s3_client = boto3.client('s3', region)
bedrock_client = boto3.client('bedrock-runtime', region)
bedrock_agent_runtime_client = boto3.client("bedrock-agent-runtime", region)

## Knowledge Base Retrieval Tool

Initialize the Bedrock Knowledge Base retrieval tool that agents will use to access examples of high-quality PR articles. This tool:
- Connects to the knowledge base created above
- Returns the top 3 most relevant examples based on semantic similarity
- Provides context for agents to understand PR writing style and structure

In [6]:
from crewai import Agent, Task, Crew, LLM
from crewai_tools.aws.bedrock.knowledge_base.retriever_tool import BedrockKBRetrieverTool

# Initialize the tool
kb_tool = BedrockKBRetrieverTool(
    knowledge_base_id=lab5_kb_id,
    number_of_results=3
)

kb_tool.description = """Retrieves examples of pristine, high quality PR articles that have been published in the past."""

## Custom Tools Definition

Define custom tools that agents can use to access guidelines and best practices. These tools read from local files containing editorial standards and writing guidelines.

In [6]:
from crewai.tools import tool

@tool("PR article review guidelines")
def pr_review_guidelines() -> str:
    """Returns the editorial guidelines for a PR article."""
    with open("pr_review_guidelines.txt", "r") as f:
        guidelines = f.read()
        return guidelines

# PR Writing Agent

The PR Writing Agent is responsible for creating initial article drafts based on provided outlines. This agent:
- Uses Amazon Nova Premier for high-quality text generation
- Has access to the knowledge base for style reference
- Can retrieve writing best practices
- Creates well-structured PR articles in markdown format

## PR Writer Agent Configuration

Configure the PR Writer agent with:
- **LLM**: Amazon Nova Premier for high-quality content generation
- **Tools**: Knowledge base retrieval and writing best practices
- **Role**: Professional media content writer specializing in PR articles
- **Task**: Generate complete PR articles from outlines or rewrite based on feedback

In [11]:
pr_writing_task_description = """You are given an outline to write a PR atricle. 
You have access to a knowledge base tool that contains some examples of pristine, high quality PRs that have been published in the past. 

- You should use the knowledge base to help with the style and structure for the article.
- You are to follow the followoing best practices for writing the article:

### Best Practices

1. Headline: Capture Attention & Key Details - Be Concise & Informative: Use active verbs like "announces," "debuts," or "unveils." Include the title, release date/platform, and a hook (e.g., talent or franchise ties).
Example: "A24 Reveals Trailer for Horror Thriller ‘Nightfall’ Starring Florence Pugh, Premiering October 2024."
Highlight Exclusives: If applicable, note premieres at festivals (e.g., "Sundance 2025 Official Selection").

2. Lead Paragraph: The 5 Ws - Summarize Who (studio, talent), What (title, genre), When (release date), Where (platforms/theaters), and Why (unique angle, legacy, or cultural relevance).
Example: "Marvel Studios’ ‘Avengers: Legacy,’ directed by Ryan Coogler, will premiere in theaters globally on May 3, 2025, marking the MCU’s first reboot of the iconic franchise."

3. Key Content Elements - Synopsis: Offer a 1–2 paragraph teaser without spoilers. Emphasize uniqueness (e.g., "a dystopian love story set in 2140’s AI-dominated society").
Talent & Production Credentials: Highlight A-list actors, acclaimed directors, or award-winning crews. Mention prior successes (e.g., "From the Oscar-winning producer of Parasite").
Behind-the-Scenes (BTS) Insights: Share filming locations, technical innovations, or adaptations (e.g., "Shot in Iceland using cutting-edge VR cinematography").

4. Quotes: Add Authenticity - Source quotes from directors, leads, or producers that convey passion or vision.
Example: "‘This film redefines resilience,’ says director Jordan Peele. ‘We’re blending horror with social commentary in ways audiences haven’t seen.’"

5. Multimedia & Links - Embed or link to trailers, stills, and BTS content. Provide access to a password-protected media kit with HD assets.
Include official social handles and hashtags (e.g., #NightfallMovie).

6. Tailor for Target Audiences - Trade Outlets (Deadline, Variety): Focus on box office prospects, production budgets, or industry trends.
Lifestyle/Cultural Press (Vulture, Rolling Stone): Highlight themes, fashion, or soundtracks.
Local Media: Tie in filming locations or local talent.

7. SEO & Discoverability - Use keywords: Title, cast names, genre, and phrases like "upcoming sci-fi movies 2025."
Link to official sites or prior related content (e.g., a franchise’s timeline).

8. Logistics & Contact Info - Clearly state release platforms (e.g., "Streaming exclusively on Netstream starting November 12").
List premiere events or fan screenings.
Provide media contacts: Name, email, phone, and PR firm.

9. Professional Polish - Proofread: Ensure dates, names, and titles are error-free.
Tone: Match the project’s vibe (e.g., playful for comedies, sleek for action).
Style Guide: Adhere to outlet-specific rules (e.g., AP style).

10. Follow-Up & Amplification - Pitch Exclusives: Offer early access to trailers or interviews for top-tier outlets.
Post-Release: Monitor coverage, engage on social media, and update with new assets (e.g., poster drops).

###


Here's the article outline:

{article_outline}


"""

# Configure Bedrock LLM
pr_writer_llm = LLM(
    model="bedrock/us.amazon.nova-premier-v1:0")

# Create agents
pr_writer_agent = Agent(
    role='PRWriter',
    goal='Write a PR article from an outline, or rewriting an article based on some feedback from a reviewer.',
    backstory='You are a professional media content writer. You are good at writing PR articles for an upcoming show/movie based on an outline of the article',
    llm = pr_writer_llm,
    tools=[kb_tool],
    verbose=True,
    max_iter=max_iterations
)

pr_writing_task = Task(
    description=pr_writing_task_description,
    expected_output="""A fully written article in markdown format""",
    async_execution=False)

# PR Reviewer Agent

The PR Reviewer Agent acts as a quality control specialist that:
- Reviews articles against established editorial guidelines
- Uses Amazon Nova Pro for analytical capabilities
- Provides detailed feedback for improvement
- Determines when quality standards are met

This agent implements the "reflection" pattern by critically evaluating the writer's output and providing constructive feedback for refinement.

In [ ]:
# Configure Bedrock LLM
pr_reviewer_llm = LLM(
    model="bedrock/us.amazon.nova-pro-v1:0")

# Create agents
pr_review_agent = Agent(
    role='PRReviewer',
    goal='Provide feedback to improve the given PR article reviewer',
    backstory='You are a professional media content analyst. You are an expert in reviewing PR articles to ensure the specific guidelines and standards are met.',
    llm = pr_reviewer_llm,
    verbose=True,
    max_iter=max_iterations
)

pr_reviewer_task_description = """Performs thorough article review to ensure specific guidelines and quality are met.

The reviewer guideline are as followed:

1. Write in clear, crisp sentences. - long or confusing sentences as clunky when they're difficult for readers to understand.

Here are the tips on to identify unclear sentences:

- Sentences longer than 40 words or spanning more than two lines
- Sentences that require multiple readings to understand
- Sentences with repeated words, especially "and" and "to"

2. Be specific. Don't leave anything open to interpretation or leave your reader guessing. 

Here are some clues your sentence isn't specific enough: 
-  it contains qualitative or subjective adjectives and adverbs (like "might," "often," or "may be"), 
- it leaves the reader asking questions.

Here are some ways to fix this problem: 
- clarify words or phrases that could be interpreted differently by different people, 
- use quantitative adjectives, like "3X faster" or "50% more time" where possible, and/or 

3. Write for a non-technical audience. Anyone should be able to understand your message without additional background information. Readers will often be left with questions if you've included concepts or undefined acronyms.

Here are some clues your sentence might be too technical: 
- your reader is left with questions, 
- it contains abbreviations or acronyms that aren't defined, and/or 
- it describes technology that an average person would not understand.

Here are some ways to fix this problem: 
3a. Include a short description of the show;
3b. define acronyms if used in the article

4. Proactively explain what makes a new show/movie interesting from similar existing show/movie. If there is any chance a reader might ask, "Isn't this movie similar to X?" then you need to address it. Avoid simply listing plots of the new movie.

Here are some ways to fix this problem:
- If there is an existing movie/show with similar plots as the new movie, address how they are different.

5. Connect the dots in a story.  You should make it clear enough how plots are related.

Here are some clues you're not connecting the dots: 
- your reader doesn't understand the plots you're writing about and/or 

Here is a way to fix this problem: 
Similar to unfolding the story, Have someone who hasn't heard of the story read the passage and ask them if it's clear to them and how everything is related.

6. Be consistent in names and descriptions. Don't confuse your reader by switching between different names or introducing additional characters near the end of your PR. Stick to one name and don't stray from the main plots.

Only return the feedback and nothing else, put the feedback in the <feedback> XML tag.
"""

pr_review_task = Task(
    description=pr_reviewer_task_description,
    expected_output="""Detailed feedback on how to improve the given article. If the article is good enough, reply with 'quality met. Good to go!' """,
    async_execution=False,
    context=[pr_writing_task])



# PR Rewriting Agent

The PR Rewriting Agent specializes in refining articles based on reviewer feedback. This agent:
- Takes the original article and reviewer feedback as input
- Uses Amazon Nova Premier for high-quality rewriting
- Incorporates specific suggestions while maintaining article coherence
- Produces improved versions that address identified issues

This completes the reflection loop: Write → Review → Rewrite → Review (if needed)

In [14]:
# Configure Bedrock LLM
pr_rewriter_llm = LLM(
    model="bedrock/us.amazon.nova-premier-v1:0")
# Create agents
pr_rewriter_agent = Agent(
    role='PRRewriter',
    goal='Rewrite an article based on the given feedback.',
    backstory="""You are a professional media content writer. You are good at writing PR articles for an upcoming show/movie based on an outline of the article.
You are to rewrite the PR article by incorporating the given review feedback.""",
    llm = pr_rewriter_llm,
    verbose=True,
    max_iter=max_iterations
)

pr_rewriting_task_description = """Rewrite the PR article by incorporating the given review feedback."""
pr_rewriting_task = Task(
    description=pr_rewriting_task_description,
    expected_output="""A fully written article in markdown format""",
    async_execution=False,
    context=[pr_writing_task, pr_review_task])

# Supervisor Agent

The Supervisor Agent orchestrates the entire workflow using CrewAI's hierarchical process. This agent:
- **Manages workflow**: Coordinates between writer, reviewer, and rewriter agents
- **Controls iterations**: Limits the write-review-rewrite cycle to prevent infinite loops
- **Uses Claude Haiku**: Optimized for fast decision-making and coordination tasks
- **Delegation authority**: Can assign tasks to appropriate specialist agents

The supervisor implements a structured approach:
1. First, delegate writing task to PR Writer
2. Then, delegate review task to PR Reviewer
3. If feedback suggests improvements, delegate rewriting to PR Rewriter
4. Limit iterations to prevent endless refinement cycles
5. Return final polished article

In [15]:
supervisor_agent_backstory = """You are a PR article orchestration agent. Your ONLY role is to manage the workflow by activating the appropriate agent at the appropriate time. You are NOT responsible for evaluating the quality or content of any agent's work.

Here are the list of agents available to you: 

1. PRWriter - Writes a PR article based on the given article outline.

2. PRReviewer - Reviews the PR article generated by the PR Writer to provide recommendation to improve the PR article.

3. PRRewriter - Rewrite a PR article based on the given feedback

You job is to delegate the writng, reviewing and rewriting tasks to the given tools. 

Here are the IMPORTANT guidelines:
- You must first write a draft article using the writing tool before performing any reviews. 
- Keep a count of the number of iterations of writing and review. You must only iterate the PR writing and review for maximum 2 times. You must not do this for more than 2 times. 
- The final draft of the PR must be a complete version formatted in markdown in the final response. Do not provide any explanation in the final response, return only the final PR article in markdown format.

"""

# Configure Bedrock LLM
supervisor_agent_llm = LLM(
    model="bedrock/us.anthropic.claude-3-5-haiku-20241022-v1:0",
    temperature=0.1,
    top_p=0.99)

# Define the manager agent
supervisor_agent = Agent(
    role="PR article orchestration manager",
    goal="Efficiently orchestrates the crew to ensure high-quality PR article creation",
    backstory=supervisor_agent_backstory,
    allow_delegation=True,
    llm=supervisor_agent_llm,
    max_iter=max_iterations
)


## Crew Assembly and Configuration

Assemble all agents and tasks into a CrewAI crew with hierarchical process management. The crew configuration:

- **Process**: Hierarchical - supervisor agent manages task delegation
- **Agents**: Writer, Reviewer, Rewriter (managed by Supervisor)
- **Tasks**: Writing, Review, Rewriting (executed in coordinated sequence)
- **Verbose**: Enabled for detailed execution logging

The hierarchical process ensures proper task sequencing and quality control through the reflection pattern.

In [16]:

crew = Crew(
    agents=[pr_review_agent, pr_rewriter_agent, pr_writer_agent],
    tasks=[pr_writing_task, pr_review_task, pr_rewriting_task],
    manager_agent=supervisor_agent,
    process=Process.hierarchical,
    verbose=True)

## Example Input: Movie PR Article Request

Define a sample movie outline to test the agentic workflow. This example includes:
- **Title & Tagline**: Core branding elements
- **Genre & Release**: Key marketing information
- **Logline**: Compelling story summary
- **Production Details**: Cast, director, budget, locations
- **Technical Elements**: Soundtrack, special features

This comprehensive outline will be processed through the write-review-rewrite cycle to produce a polished PR article.

In [17]:
query = """Write a PR article for an upcoming movie. Here's an outline: 

Title: Midnight Vendetta
Tagline: “When the clock strikes twelve, justice wears no mask.”
Genre: Action/Thriller
Release Date: November 22, 2025 (Theatrical & IMAX)

Logline:
A disgraced former MI6 agent infiltrates a glittering Dubai masquerade ball to dismantle a trillion-dollar cyberweapons syndicate—but must confront his deadliest enemy: the traitor who framed him for murder.

Key Production/Cast Details:
Director: David Leitch (Atomic Blonde, Bullet Train)—promises “brutally elegant fight choreography blending Bourne-style close combat with Dubai’s opulent settings.”
Studio: Pika Pictures (Budget: $200M)
Filming Locations: Dubai’s Burj Khalifa, Palm Jumeirah, and a custom-built 360-degree rotating ballroom set for the climactic fight.
Soundtrack: Pulse-pounding score by Ludwig Göransson 
"""

inputs = {
    'article_outline': query
    }

## Execute the Agentic Workflow

Launch the CrewAI workflow with the movie outline. The supervisor agent will:
1. **Delegate writing task** to PR Writer agent
2. **Delegate review task** to PR Reviewer agent
3. **Coordinate rewriting** if improvements are needed
4. **Manage iterations** to ensure quality while preventing infinite loops
5. **Return final article** in markdown format

The `kickoff()` method initiates the hierarchical process with the provided inputs.

In [ ]:
pr_article_generation_result = crew.kickoff(inputs=inputs)

## Display the Final Result

Print the final PR article generated by the multi-agent system. This result represents the output of the complete reflection workflow:
- **Initial draft** created by the PR Writer
- **Quality review** performed by the PR Reviewer
- **Refinements** applied by the PR Rewriter (if needed)
- **Final approval** coordinated by the Supervisor

The result should be a polished, professional PR article in markdown format that meets all editorial guidelines and quality standards.

In [ ]:
print(pr_article_generation_result)

## Summary

This notebook demonstrates a sophisticated multi-agent system using CrewAI that implements the reflection pattern for automated content generation. Key achievements:

### Architecture Benefits
- **Specialized Agents**: Each agent has a specific role (writing, reviewing, rewriting)
- **Quality Control**: Built-in review and refinement cycles
- **Scalable Workflow**: Hierarchical management enables complex task coordination
- **AWS Integration**: Leverages Bedrock LLMs and Knowledge Bases for enhanced capabilities

### Technical Implementation
- **Multiple LLM Models**: Different models optimized for different tasks
- **Knowledge Retrieval**: Context-aware content generation using example articles
- **Iteration Control**: Prevents infinite refinement loops
- **Tool Integration**: Custom tools for guidelines and best practices

### Use Cases
This pattern can be extended to other content generation scenarios:
- Marketing copy creation and review
- Technical documentation writing
- Creative content development
- Quality assurance workflows

The reflection pattern ensures high-quality outputs through systematic review and refinement, making it ideal for professional content creation workflows.